In [1]:
import pandas as pd
from utils.file_utils import read_json_tree

In [2]:
def parse_file_path(file_path: str) -> tuple[str,str,int]:
    file_name = file_path.split("/")[-1]
    info = file_name.split("_")
    solver_name = info[0]
    tree_name = "_".join(info[1:4])
    dimension = int(info[4])
    return solver_name, tree_name, dimension

In [3]:
def restric_range(full_log:pd.DataFrame,population_size:int, staring_generation:int, generation_range:int) -> pd.DataFrame:
    starting_index = staring_generation * population_size
    ending_index = starting_index + generation_range * population_size
    stat_series = full_log.iloc[starting_index:ending_index].groupby("eval_node_id").size()
    return stat_series.to_frame(name="count").reset_index()


In [4]:
file_path = "/local/pareto_benchmark/benchmark-visualizer/data/pop100_50000iter/exp_csvs/NSGAII_breadth_base_1_3_StoppingByEvaluations_2023-03-22T10-51-30.914512.csv"
solver_name, tree_name, dimension = parse_file_path(file_path=file_path)
tree = read_json_tree("experiment_trees/"+tree_name+".json")
tree["nodes"].append({"id": 0, "minima": -1.0, "symbol": []})
tree = pd.DataFrame(tree["nodes"])

In [5]:
eval_log = pd.read_csv(file_path, index_col=0)[["t", "y1", "y2", "eval_node_id", "diagonal_length", "step", "t_org", "y_org"]]
eval_log.sort_values(by="step", ascending=True,inplace=True)
eval_log.reset_index(inplace=True,drop=True)

In [6]:
population_size = 100
starting_generation = 0
generation_range = 500
stat_series = restric_range(full_log=eval_log,population_size=population_size,staring_generation=starting_generation,generation_range=generation_range)
histo = pd.merge(tree, stat_series, how="inner", left_on="id",right_on="eval_node_id")
histo = histo.sort_values(by="id").reset_index(drop=True)

In [7]:
histo

,id,minima,symbol,eval_node_id,count
0,0,-1.0,[],0,10488
1,2,-2.1,[-1],2,1
2,3,-2.2,[2],3,4
3,4,-2.3,[-2],4,39507
